# 필요한 module import 

In [1]:
import json
import requests
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
from webob.compat import urlparse
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))
from tqdm import notebook
import math
from urllib.request import urlopen
from tqdm import tqdm
import time
pd.options.display.max_colwidth = None

C:\Users\jki01\AppData\Local\Temp\ipykernel_9964\2207815226.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# URL 추출

##### url 과 뉴스 제목을 추출하는 function 정의

In [ ]:
def craw():
    # dl tag의 newsList class를 크롤링
    news = bs.find_all('dl', {'class' : 'newsList'})
    news_list = []
    news_list_2 = []
    title_list = []
    title_list_2 = []
    url_list = []
    url_list_2 = []
    
    #dl tag의 articleSubject와 articleSummary class 저장
    for new in news:
        title = new.find_all('dt',{'class' : 'articleSubject'})
        title_2 = new.find_all('dd',{'class' : 'articleSubject'})
#         date_time = new.find_all('dd',{'class' : 'articleSummary'})
        # 첫번 째 articleSubject가 저장된 title 에서 url 추출 
        for dt in title:
            dd = dt.text[1:-1]
            link = dt.find("a")["href"]
            article_url = 'https://finance.naver.com/' + link
            url_list.append(article_url)
            title_list.append(dd)
        # 두번 째 articleSubject가 저장된 title_2에서 news 제목 추출
        for oo in title_2:
            article_title = oo.text[1:-1]
            link = oo.find("a")["href"]
            article_url = 'https://finance.naver.com/' + link
            text_list = [article_title, article_url]
            url_list_2.append(article_url)
            title_list_2.append(article_title)
        # 추출한 요소들을 list 형태로 저장해서 url_result 와 title_result로 return
        url_result = [url_list, url_list_2]
        title_result = [title_list, title_list_2]
    return url_result, title_result

##### 검색 keyword | 시작날짜 | 종료날짜 설정 후 해당하는 url과 뉴스 제목 크롤링

In [ ]:
# 검색, 시작날짜, 종료날짜 변수로 저장
search = "삼성전자"
stDateStart= "2017-01-01"
stDateEnd= "2022-04-26"
#검색어 인코딩(아스키 오류를 방지하기 위해)
euc_data = search.encode('euc-kr')
tmp = str(euc_data).replace("\\x","%")[2:-1]
#여러 페이지 크롤링
page = 1
url = f"https://finance.naver.com/news/news_search.naver?rcdate=1&q={tmp}&x=0&y=0&sm=all.basic&pd=4&stDateStart={stDateStart}&stDateEnd={stDateEnd}"
req = requests.get(url)
bs = BeautifulSoup(req.content, "html.parser")
strong = bs.find_all('strong')
article_num = strong[-3]
article_num = int(re.sub(r"[^a-zA-Z0-9]","",article_num.text))
i = math.ceil(article_num/20)
tmp3 = []
print(article_num) # 검색 했을 때 총 몇개의 기사가 검색되는지 표시
i += 1
for page in tqdm(range(1,i)):
    url = f"https://finance.naver.com/news/news_search.nhn?rcdate=&q={tmp}\&x=5&y=7&sm=all.basic&pd=1&{stDateStart}&{stDateEnd}&page={page}"
    req = requests.get(url)
    bs = BeautifulSoup(req.content, "html.parser")
    # 기사 갯수 파악 후 for문 범위 정하기
    url_list, title_list = craw()
    url_list = [y for x in url_list for y in x]
    title_list = [y for x in title_list for y in x]
    for j in range(len(url_list)):
        tmp2 = []
        tmp2.append(url_list[j])
        tmp2.append(title_list[j])
        tmp3.append(tmp2)
#     print(url)
tmp3 = pd.DataFrame(tmp3)
tmp3

##### 한 페이지당 뉴스가 20개씩 검색 됩니다.
##### 크롤링한 페이지가 잘못될 경우 추출한 url 20개가 삼성전자가 아닌 ΜοΦΚάϋάΎ, ЛяМКРќРк, їпЉЇјьјЏ로 잘못 추출되었습니다.
##### 아래의 코드로 어떤 것들이 잘못되었는지 전체적으로 확인 가능 합니다.

In [ ]:
# 추출한 url 을 20개씩 출력
# for i in tqdm(range(int(len(tmp3) / 20))):
#     print(tmp3[0][i*20])

##### 잘못된 url과 올바른 url을 구분해서 변수에 저장

In [ ]:
n_error_url = test_samples[test_samples['0'].str.contains('삼성전자') == True]
error_url = test_samples[test_samples['0'].str.contains('삼성전자') == False]

##### 잘못된 url을 '삼성전자'로 올바르게 변경

In [ ]:
# 대략적으로 어떻게 오류가 났는지 확인 할 수 있는 코드입니다.
# error_url

In [ ]:
# error_url에서 'ЛяМКРќРк'을 '삼성전자'로 변경후 error_v1에 저장 
error_v1 = []
for i in error_url['0']:
    error_list = i.replace('ЛяМКРќРк','삼성전자')
    error_v1.append(error_list)

In [ ]:
# error_v1 'їпЉЇјьјЏ'을 '삼성전자'로 변경후 error_v2에 저장 
error_v2 = []
for i in error_v1:
    error_list = i.replace('їпЉЇјьјЏ', '삼성전자')
    error_v2.append(error_list)

In [ ]:
# error_v2 'ΜοΦΚάϋάΎ'을 '삼성전자'로 변경후 error_v3에 저장 
error_v3 = []
for i in error_v2:
    error_list = i.replace('ΜοΦΚάϋάΎ', '삼성전자')
    error_v3.append(error_list)

In [ ]:
# 삼성전자가 안들어간 url이 있는지 다시 확인해 줍니다.
err_check = pd.DataFrame(error_v3)
err_check[err_check[0].str.contains('삼성전자') == False]

##### 수정된 url과 올바른 url을 다시 합쳐줍니다.

In [ ]:
samsung_url = pd.concat([n_error_url['0'], err_check])

In [ ]:
samsung_url

##### 추출한 data를 CSV 형태로 저장 (아스키 오류를 방지하기 위해)

In [ ]:
samsung_url.to_csv('samsung_url.csv')

In [ ]:
test_samples = pd.read_csv('samsung_url.csv', index_col=0, encoding='utf-8')

In [ ]:
crawling = test_samples['0']

In [ ]:
# url 데이터에 손실이 있는지 확인 합니다.
print(crawling)
print(len(crawling))

# 뉴스 내용 크롤링

In [ ]:
# 변수 초기화
test = []
test_final = []

# 삼성전자 encoding
search='삼성전자'
search_euc = search.encode('euc-kr')
search_ = str(search_euc).replace("\\x","%")[2:-1]

In [ ]:
for i in tqdm(crawling):
    i = re.sub(r'[ㄱ-ㅣ가-힣]',search_, i)
    page = urlopen(i)
    soup = BeautifulSoup(page, 'html.parser', from_encoding='cp-949')
    news_contents = soup.find('div', id='content', class_='articleCont')
    news_data = soup.find('span', class_ = 'article_date')
    
    contents_list = list(news_contents)[2:-2]
    contents_list_to_string = ' '.join(str(c) for c in contents_list)
    contents_list_to_string_strip = contents_list_to_string.strip(' \t\n\r' )
    contents_list_to_string_strip_pattern = re.sub(pattern = '<[^>]*>', repl = '', string = contents_list_to_string)
    
    test=[]
    test.append(contents_list_to_string_strip_pattern)
    test.append(news_data.text)
    test_final.append(test)
    

##### 최종 파일을 DataFrame으로 만든 후 CSV 형태로 저장 합니다.

In [ ]:
samsung_news = pd.DataFrame(test_final)
samsung_news

In [ ]:
samsung_news.to_csv('samsung_news.csv')